# В этом ноутбуке представлена вторая гипотеза для оценки сходства датасетов, а также реализован метод сравнения датасетов в соответствии со второй гипотезой.

**Вторая гипотеза** для метода оценки сходства датасетов:

_Сходство датасетов может быть связано со схожестью их текстового содержания, которое отражается в близких средних значениях эмбеддингов._

-------------------

Поэтому **метод сравнения датасетов** следующий:

Cделать векторные представления от каждого текста из датасета (например, с помощью bert) и усреднить. Для разных датасетов сравнить полученный эмбеддинг, например, с помощью cosine_similarity. Чем больше полученное значение, тем более похожими являются полученные усредненные эмбеддинги и, по нашей гипотезе, тем больше сходство сравниваемых датасетов.

In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.metrics.pairwise import cosine_similarity
drive.mount('/content/drive')
!pip install simpletransformers
from simpletransformers.language_representation import RepresentationModel

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
model = RepresentationModel(
        model_type="bert",
        model_name="bert-base-uncased",
        use_cuda=False
    )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTextRepresentation: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
hotel_reviews = 'hotel reviews'
movie_reviews = 'movie reviews'
spam_sms = 'spam sms'
spam_emails = 'spam emails'

datasets_names = [hotel_reviews, movie_reviews, spam_sms, spam_emails]

In [4]:
def get_dataset_in_correct_form(dataset_name):
    if dataset_name == spam_sms:
        df = pd.read_csv('/content/drive/MyDrive/data_for_colab/spam_sms.csv', encoding = "ISO-8859-1")
        df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)
        df.columns = ['IS_SPAM', 'DATA_COLUMN']
        df['IS_SPAM'] = (df['IS_SPAM'] == 'spam').astype(int)
        df_positive = df[df['IS_SPAM']==1]
        df_negative = df[df['IS_SPAM']==0]
        # Тестовая выборка
        n_test = df_negative.shape[0] // 2
        df_negative_test = df_negative.tail(n_test)
        n_test = df_positive.shape[0] // 2
        df_positive_test = df_positive.tail(n_test)
        df_balanced_test = pd.concat([df_negative_test, df_positive_test])
        # Обучающая выборка
        n_train = df_negative.shape[0] // 2
        df_negative_train = df_negative.head(n_train)
        n_train = df_positive.shape[0] // 2
        df_positive_train = df_positive.head(n_train)
        df_balanced_train = pd.concat([df_negative_train, df_positive_train])

    elif dataset_name == spam_emails:
        df = pd.read_csv('/content/drive/MyDrive/data_for_colab/spam_emails.csv', encoding = "ISO-8859-1")
        df.drop(columns=['Unnamed: 0', 'label'], inplace=True)
        df.columns = ['DATA_COLUMN', 'IS_SPAM']
        df['DATA_COLUMN'] = df['DATA_COLUMN'].apply(lambda x: x.replace('\r\n', ' ').replace('\n', ' '))
        df_positive = df[df['IS_SPAM']==1]
        df_negative = df[df['IS_SPAM']==0]
        # Тестовая выборка
        n_test = df_negative.shape[0] // 2
        df_negative_test = df_negative.tail(n_test)
        n_test = df_positive.shape[0] // 2
        df_positive_test = df_positive.tail(n_test)
        df_balanced_test = pd.concat([df_negative_test, df_positive_test])
        # Обучающая выборка
        n_train = df_negative.shape[0] // 2
        df_negative_train = df_negative.head(n_train)
        n_train = df_positive.shape[0] // 2
        df_positive_train = df_positive.head(n_train)
        df_balanced_train = pd.concat([df_negative_train, df_positive_train])

    elif dataset_name == hotel_reviews:
        df = pd.read_csv('/content/drive/MyDrive/data_for_colab/tripadvisor_hotel_reviews.csv')
        df = df[df.Rating != 3]
        df['is_positive'] = (df['Rating'] >= 4).astype(int)
        df.drop(columns=['Rating'], inplace=True)
        df.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
        df_positive = df[df['LABEL_COLUMN']==1]
        df_negative = df[df['LABEL_COLUMN']==0]
        # Тестовая выборка
        n_test = (df_negative.shape[0] // 4) * 3
        df_negative_test = df_negative.tail(n_test)
        n_test = (df_positive.shape[0] // 20) * 3
        df_positive_test = df_positive.tail(n_test)
        df_balanced_test = pd.concat([df_negative_test, df_positive_test])
        # Обучающая выборка
        n_train = df_negative.shape[0] // 4
        df_negative_train = df_negative.head(n_train)
        n_train = df_positive.shape[0] // 20
        df_positive_train = df_positive.head(n_train)
        df_balanced_train = pd.concat([df_negative_train, df_positive_train])
    
    elif dataset_name == movie_reviews:
        df = pd.read_csv('/content/drive/MyDrive/data_for_colab/IMDB Dataset.csv')
        df['is_positive'] = (df['sentiment'] == 'positive').astype(int)
        df.drop(columns=['sentiment'], inplace=True)
        df.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
        df_positive = df[df['LABEL_COLUMN']==1]
        df_negative = df[df['LABEL_COLUMN']==0]
        # Для тестовой выборки берем последние 10% негативных отзывов и последние 10% позитивных отзывов
        n_test = df_negative.shape[0] // 10 # в оригинале df_negative.shape[0] // 10
        df_negative_test = df_negative.tail(n_test)
        n_test = df_positive.shape[0] // 10 # df_positive.shape[0] // 10
        df_positive_test = df_positive.tail(n_test)
        df_balanced_test = pd.concat([df_negative_test, df_positive_test])
        # Для обучающей выборки берем первые 2.5% из начала датасета.
        n_train = df_negative.shape[0] // 40 # в оригинале df_negative.shape[0] // 40
        df_negative_train = df_negative.head(n_train)
        n_train = df_positive.shape[0] // 40 # в оригианале df_positive.shape[0] // 40
        df_positive_train = df_positive.head(n_train)
        df_balanced_train = pd.concat([df_negative_train, df_positive_train])

    else:
        raise ValueError('Wrong dataset name')

    X_train = df_balanced_train['DATA_COLUMN'].squeeze()
    X_test = df_balanced_test['DATA_COLUMN'].squeeze()
    # dataset_in_correct_form = list(pd.concat([X_train, X_test])) # работает слишком долго
    dataset_in_correct_form = list(pd.concat([X_train, X_test]))[:100]
    return dataset_in_correct_form

In [5]:
def get_vector_representations(dataset_in_correct_form):
    items_vectors = model.encode_sentences(dataset_in_correct_form, combine_strategy="mean")
    return items_vectors

In [6]:
# cosine_similarity(np.mean(get_vector_representations(get_dataset_in_correct_form(spam_emails)), axis = 0).reshape(1, -1), np.mean(get_vector_representations(get_dataset_in_correct_form(spam_sms)), axis = 0).reshape(1, -1))[0][0]

На 1/16 часть исследования требуется 2 минуты, если брать первые 100 записей в датасете.  Если брать весь датасет, тогда на 1/16 часть исследования надо более получаса, и я завершил выполнение кода досрочно. 

In [7]:
df_with_second_method_results = pd.DataFrame(columns=[hotel_reviews, movie_reviews, spam_sms, spam_emails])
_i = 0
for cur_first_dataset in datasets_names:
    for cur_second_dataset in datasets_names:
        _i += 1
        print(_i, 'out of', len(datasets_names) ** 2, ':', "Please, be patient! Working on comparing", cur_first_dataset, 'with', cur_second_dataset)
        df_with_second_method_results.loc[cur_first_dataset, cur_second_dataset] = cosine_similarity(np.mean(get_vector_representations(get_dataset_in_correct_form(cur_first_dataset)), axis = 0).reshape(1, -1),
                                                                                                     np.mean(get_vector_representations(get_dataset_in_correct_form(cur_second_dataset)), axis = 0).reshape(1, -1))[0][0]

1 out of 16 : Please, be patient! Working on comparing hotel reviews with hotel reviews
2 out of 16 : Please, be patient! Working on comparing hotel reviews with movie reviews
3 out of 16 : Please, be patient! Working on comparing hotel reviews with spam sms
4 out of 16 : Please, be patient! Working on comparing hotel reviews with spam emails
5 out of 16 : Please, be patient! Working on comparing movie reviews with hotel reviews
6 out of 16 : Please, be patient! Working on comparing movie reviews with movie reviews
7 out of 16 : Please, be patient! Working on comparing movie reviews with spam sms
8 out of 16 : Please, be patient! Working on comparing movie reviews with spam emails
9 out of 16 : Please, be patient! Working on comparing spam sms with hotel reviews
10 out of 16 : Please, be patient! Working on comparing spam sms with movie reviews
11 out of 16 : Please, be patient! Working on comparing spam sms with spam sms
12 out of 16 : Please, be patient! Working on comparing spam sms

In [8]:
df_with_second_method_results

,hotel reviews,movie reviews,spam sms,spam emails
hotel reviews,1.0,0.767404,0.592511,0.782914
movie reviews,0.767404,1.0,0.729809,0.862794
spam sms,0.592511,0.729809,1.0,0.720972
spam emails,0.782914,0.862794,0.720972,1.0


In [9]:
df_with_second_method_results.to_csv('/content/drive/MyDrive/data_for_colab/dataframes/second_approach/df_with_second_method_results.csv')


Теперь будем искать корреляцию с разницей качества из раздела "Дополнительное исследование"